<a href="https://colab.research.google.com/github/cam2149/MachineLearningV/blob/main/taller/EDAv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Equipo**

- Nicolás Colmenares

- Carlos Martinez

1. Análisis Exploratorio de Datos (EDA) - 0.5 pts

  -  Carga y limpieza de los datos.

  - Visualización de tendencias y patrones de dengue.

  -  Análisis de correlaciones entre variables.

# 0. Configuraciones de Colab

Mover Kaggle.json a la ubicación correcta después de subirlo

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Descargar dataset de la competencia

In [ ]:
!kaggle competitions download -c aa-v-2025-i-pronosticos-nn-rnn-cnn

In [ ]:
!mkdir /content/kaggle/output
!mkdir /content/kaggle/input

In [ ]:
!mv aa-v-2025-i-pronosticos-nn-rnn-cnn.zip /content/kaggle/input

In [ ]:
!unzip /content/kaggle/input/aa-v-2025-i-pronosticos-nn-rnn-cnn.zip -d /content/kaggle/input/

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
#/kaggle/input
import os
for dirname, _, filenames in os.walk('/content/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib as mpl
import matplotlib.pyplot as plt   # data visualization
import seaborn as sns             # statistical data visualization
from matplotlib.colors import LinearSegmentedColormap

# 2. Configs

In [ ]:
config = {
    "TRAIN_DIR": '/content/kaggle/input/df_train.parquet',
    "TEST_DIR": '/content/kaggle/input/df_test.parquet',
    "SUBMISSION_DIR": '/content/sample_submission.csv'
}

# Exploración

## Diccionario

train.parquet - El conjunto de datos de entrenamiento
test.parquet - El conjunto de datos de prueba
sample_submission.csv - un ejemplo de un archivo a someter en la competencia

| **Variable**         | **Descripción**                                                                                      |
|-----------------------|------------------------------------------------------------------------------------------------------|
| id_bar               | identificador único del barrio                                                                      |
| anio                 | Año de ocurrencia                                                                                   |
| semana               | Semana de ocurrencia                                                                               |
| Estrato              | Estrato socioeconómico del barrio                                                                   |
| area_barrio          | Área del barrio en km²                                                                             |
| dengue               | Conteo de casos de dengue                                                                          |
| concentraciones      | Cantidad de visitas e intervención a lugares de concentración humana (Instituciones)                |
| vivienda             | Conteo de las visitas a viviendas a revisión y control de criaderos                                 |
| equipesado           | Conteo de las fumigaciones con Maquinaria Pesada                                                   |
| sumideros            | Conteo de las intervenciones a los sumideros                                                       |
| maquina              | Conteo de las fumigaciones con motomochila                                                         |
| lluvia_mean          | Lluvia promedio en la semana i                                                                     |
| lluvia_var           | Varianza de la lluvia en la semana i                                                               |
| lluvia_max           | Lluvia máxima en la semana i                                                                       |
| lluvia_min           | Lluvia mínima en la semana i                                                                       |
| temperatura_mean     | Temperatura promedio en la semana i                                                                |
| temperatura_var      | Varianza de la temperatura en la semana i                                                          |
| temperatura_max      | Temperatura máxima en la semana i                                                                  |
| temperatura_min      | Temperatura mínima en la semana i                                                                  |


## Lectura del dataset de entrenamiento

In [ ]:
# Assuming 'series_test.parquet' is in the current directory or you provide the correct path
try:
    train_df = pd.read_parquet(config["TRAIN_DIR"]).iloc[:,1:]
    print(train_df.describe())
    print(train_df.info())
    print(train_df.shape)
except FileNotFoundError:
    print("Error: 'series_train.parquet' not found. Please make sure the file exists in the current directory or provide the correct path.")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
train_df.head()

In [ ]:
train_df.dtypes

Todas las columnas son númericas

In [ ]:
train_df.describe().T

Estrato va del 1 al 3

Los casos de dengue llegan a 29

lluvia min siempre es 0

In [ ]:
missing_values = train_df.isnull().sum()
missing_values[missing_values > 0]

No hay valores nulos

## Visualizar la serie temporal

In [ ]:
def plot_df(df, x, y, title="", xlabel='Year_Week', ylabel='Cases of Dengue', dpi=100):
    plt.figure(figsize=(15,4), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()


In [ ]:
# Create a new 'anio_semana' column
train_df['anio_semana'] = train_df['anio'].astype(str) + '-' + train_df['semana'].astype(str).str.zfill(2)
train_df['anio_semana'] = pd.to_datetime(train_df['anio_semana'] + '-1', format='%Y-%W-%w')

# Get unique 'id_bar' values and sort them
unique_id_bars = sorted(train_df['id_bar'].unique())

# Calculate number of rows needed for 1 column (which is the same as the number of unique id_bars)
num_rows = len(unique_id_bars)

# Create a figure and a grid of subplots with 1 column
fig, axes = plt.subplots(num_rows, 1, figsize=(10, 5 * num_rows))

# Iterate and plot on each subplot
for i, id_bar in enumerate(unique_id_bars):
    id_bar_data = train_df[train_df['id_bar'] == id_bar]
    axes[i].plot(id_bar_data['anio_semana'], id_bar_data['dengue'])  # Use axes[i] for single column
    axes[i].set_xlabel('Año-Semana')
    axes[i].set_ylabel('Casos de Dengue')
    axes[i].set_title(f'id_bar: {id_bar}')

# Adjust spacing between subplots
plt.tight_layout()
plt.show()

In [ ]:
# Create a new 'anio_semana' column
train_df['anio_semana'] = train_df['anio'].astype(str) + '-' + train_df['semana'].astype(str).str.zfill(2)
train_df['anio_semana'] = pd.to_datetime(train_df['anio_semana'] + '-1', format='%Y-%W-%w')

# Get unique 'id_bar' values and assign colors
unique_id_bars = train_df['id_bar'].unique()
colors = plt.cm.get_cmap('viridis', len(unique_id_bars))  # Use a colormap

# Create a single plot
fig, ax = plt.subplots(figsize=(10, 6))

# Plot data for each 'id_bar' with different colors
for i, id_bar in enumerate(unique_id_bars):
    id_bar_data = train_df[train_df['id_bar'] == id_bar]
    ax.plot(id_bar_data['anio_semana'], id_bar_data['dengue'],
            color=colors(i), label=f'id_bar: {id_bar}')

# Customize plot
ax.set_xlabel('Año-Semana')
ax.set_ylabel('Casos de Dengue')
ax.set_title('Tendencia de Casos de Dengue por id_bar (Color)')
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))  # Place legend outside

plt.tight_layout()
plt.show()

In [ ]:
plt.bar(train_df['ESTRATO'], train_df['dengue'])
plt.xlabel('Estrato')
plt.ylabel('Casos de Dengue')
plt.title('Casos de Dengue por Estrato')
plt.show()

## Matriz de correlación

In [ ]:
corr_matrix = train_df.iloc[:,:-1].corr()

# Define colors and positions for the gradient
colors = [(1, 0, 0), (0.6, 0.8, 1), (1, 0.5, 0)]  # Red, Light Blue, Orange in RGB
positions = [0, 0.6, 1]  # Positions for color transitions (0 to 1)

# Create custom colormap
cmap = LinearSegmentedColormap.from_list('custom_cmap', list(zip(positions, colors)))

# Create heatmap with custom colormap
sns.heatmap(corr_matrix,
            yticklabels=corr_matrix.columns,
            xticklabels=False,
            annot=False,
            cmap=cmap,
            vmin=-1, vmax=1,  # Set color range limits
            center=0)  # Center color at 0


plt.gca().spines['bottom'].set_visible(False)  # Hide the x-axis line

plt.title('Matriz de Correlación')
plt.show()

Se presenta correlación entre **id_bar** y **area_barrio**, además de entre las variables de lluvia al igual que las de temperatura.

La mayor correlacción con **dengue** es **anio** y **semana**

In [ ]:
# Using statmodels: Subtracting the Trend Component
from statsmodels.tsa.seasonal import seasonal_decompose

# Check for zero and negative values in 'dengue' column
if (train_df['dengue'] <= 0).any():
    # If zero or negative values exist, use 'additive' model
    result_mul = seasonal_decompose(train_df['dengue'], model='additive', period=30)
else:
    # If no zero or negative values, use 'multiplicative' model
    result_mul = seasonal_decompose(train_df['dengue'], model='multiplicative', period=30)

detrended = train_df['dengue'].values - result_mul.trend
plt.plot(detrended)
plt.title('Dengue desestabilizados restando el componente de tendencia', fontsize=16)

In [ ]:
# Subtracting the Trend Component


# Time Series Decomposition
result_mul = seasonal_decompose(df['Number of Passengers'], model='multiplicative', period=30)


# Deseasonalize
deseasonalized = df['Number of Passengers'].values / result_mul.seasonal


# Plot
plt.plot(deseasonalized)
plt.title('Air Passengers Deseasonalized', fontsize=16)
plt.plot()

La forma habitual de comprobar la estacionalidad de una serie temporal es graficarla y comprobar si existen patrones repetibles en intervalos de tiempo fijos. Por lo tanto, el tipo de estacionalidad se determina por el reloj o el calendario.
Sin embargo, si deseamos una evaluación más precisa de la estacionalidad, utilice el gráfico de la Función de Autocorrelación (ACF). Debido a la fuerte estacionalidad, el gráfico de la ACF suele revelar picos repetidos y definidos en los múltiplos de la ventana estacional.

In [ ]:
# Test for seasonality
from pandas.plotting import autocorrelation_plot

# Draw Plot
plt.rcParams.update({'figure.figsize':(10,6), 'figure.dpi':120})
autocorrelation_plot(train_df['dengue'].tolist())